In [1]:
#CLUSTER GORDIAS LIS 
import pyTigerGraph as tg
import pandas as pd
import gradio as gr
hostName = "https://d6e9eef375704c8893b47bdc6132b082.i.tgcloud.io"
graphName = "VWG"
secret = "8uan20otpbh626fl4s449aumie58o4h9"
userName = "user_1"
password = "A1z2e3r4*"
tigergraph_insights_map = "https://tools.tgcloud.io/insights/app/qepJkoYLXfcWTB4d5ExgVT/page/4NjQNvfLhhTXHgQZBkyxtM/widgetShare/6wci2PjrXvzciTfmrH9pQY?domain=d6e9eef375704c8893b47bdc6132b082.i&orgName=lis-gordias&clusterid=507badf1-6ed3-4a79-9aaa-82489ed609ef&TigerGraphToken=e4120fbd-1fb5-4383-a727-33fb7032eb53"
graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"secret token: {authToken}")
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password, apiToken=authToken)

secret token: ug9mkc24jtimcc9j0b2qo8dkjhjfdfh3


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\pyTigerGraph\pyTigerGraphBase.py:105: DeprecationWarning: The `apiToken` parameter is deprecated; use `getToken()` function instead.
  warnings.warn(


In [2]:
# # FREE CLUSTER EDGE
hostName1 = "https://506118e7a5824411b8cff3b579d8809b.i.tgcloud.io"
graphName1 = "VWG1"
secret1 = "4alh4gsrhiq5tlfb3tf5gro5srpa5e35"
userName1 = "user_1"
password1 = "A1z2e3r4*"
tigergraph_insights_map1 = "https://tools.tgcloud.io/insights/app/gBQu5GpNcMMoNttUGH8t29/page/mwMTnWnUDwp1PgFEE5Vapi/widgetShare/veFZVtxZW1gpd4GXchssZ2?domain=506118e7a5824411b8cff3b579d8809b.i&orgName=jrlisdata-org-2024325&clusterid=95a180a6-6c60-4d83-94ff-5291bf665469&TigerGraphToken=2cf416dd-3e79-4428-a5c0-1a68168b608b"
graph1 = tg.TigerGraphConnection(host=hostName1, graphname=graphName1)

authToken1 = graph1.getToken(secret1)
authToken1 = authToken1[0]
print(f"secret token 1: {authToken1}")
conn1 = tg.TigerGraphConnection(host=hostName1, graphname=graphName1, username=userName1, password=password1, apiToken=authToken1)

secret token 1: 395kqmln948q1cf8aspr7vnd479l216i


In [3]:
# deleted = conn1.upsertEdge(sourceVertexType="Nodes",targetVertexType="Nodes",
#                         sourceVertexId="Las Palmas de Gran Canaria",targetVertexId="Seville",edgeType="distribute_to")

# Tab 0 (cargar datos)

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
atributes_nodes = {}
atributes_edges = {}

# Helper functions
def path_taken(results):
    # Extrae el conjunto de aristas desde el diccionario de resultados.
    edges = results[2]['@@display_edge_set']
    
    # Inicializa una variable para almacenar el nodo inicial.
    node_initial = None
    
    # Crea un conjunto de identificadores de nodos a los que se dirige cada arista.
    edge_to_ids = {edge['to_id'] for edge in edges}
    
    # Busca el nodo inicial (el nodo que no es el destino de ningún arista).
    for edge in edges:
        if edge['from_id'] not in edge_to_ids:
            node_initial = edge['from_id']
            break
    
    # Si no se encuentra un nodo inicial, lanza una excepción.
    if node_initial is None:
        raise ValueError("initial node cannot be determined.")
    
    # Inicializa el nodo actual como el nodo inicial y crea una lista para el camino.
    current_node = node_initial
    path = [current_node]
    
    # Recorre los bordes para construir el camino desde el nodo inicial.
    while True:
        next_node = None
        
        # Busca el siguiente nodo en el camino basado en el nodo actual.
        for edge in edges:
            if edge['from_id'] == current_node:
                next_node = edge['to_id']
                break
        
        # Si no se encuentra el siguiente nodo, termina el bucle.
        if next_node is None:
            break
        
        # Añade el siguiente nodo al camino y actualiza el nodo actual.
        path.append(next_node)
        current_node = next_node
    
    # Crea un diccionario de nodos mapeando cada nodo a sí mismo.
    nodes = {node_name: node_name for node_name in path}
    
    # Crea una lista ordenada de nodos basada en el camino encontrado.
    ordered_nodes = [nodes[node_name] for node_name in path]
    
    # Devuelve la lista de nodos ordenados que representa el camino tomado.
    return ordered_nodes


def add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, charge):
    # Crea una lista de ciudades existentes a partir de los nodos en 'nodes_batch_full'.
    existing_cities = [city for city, _ in nodes_batch_full]
    
    # Itera sobre cada nodo en el 'nodes_batch' para actualizar o agregar nodos en 'nodes_batch_full'.
    for node, _ in nodes_batch:
        if node in existing_cities:
            # Si el nodo ya existe en 'nodes_batch_full', busca el índice del nodo para actualizar el stock.
            for i, (city, stock) in enumerate(nodes_batch_full):
                if city == node:
                    # Si es el primer nodo, decrementa el stock por 'charge'.
                    # De lo contrario, incrementa el stock por 'charge'.
                    if i == 0:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] - charge})
                    else:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] + charge})
            continue
        else:
            # Si el nodo no está en 'nodes_batch_full', agrega nuevas entradas para los nodos de 'nodes_batch'.
            nodes_batch_full.extend([
                (nodes_batch[0][0], {"Stock": nodes_batch[0][1]["Stock"]}),
                (nodes_batch[1][0], {"Stock": nodes_batch[1][1]["Stock"]})
            ])
            # Retorna 'nodes_batch_full' después de agregar las nuevas entradas.
            return nodes_batch_full
    
    # Retorna 'nodes_batch_full' después de actualizar o agregar los nodos.
    return nodes_batch_full

def add_or_update_final_edges_batch(edges_batch_full, edges_batch, charge):
    # Crea una lista de bordes existentes a partir de 'edges_batch_full', considerando ambos sentidos.
    existing_edges = [(edge[0], edge[1]) for edge in edges_batch_full]
    
    # Itera sobre cada borde en 'edges_batch' para actualizar o agregar bordes en 'edges_batch_full'.
    for edge in edges_batch:
        # Verifica si el borde ya existe en 'edges_batch_full' considerando ambos sentidos.
        if (edge[0], edge[1]) in existing_edges or (edge[1], edge[0]) in existing_edges:
            # Si el borde ya existe, busca el índice del borde para actualizar el movimiento diario.
            for i, (origin, destination, movement) in enumerate(edges_batch_full):
                # Compara ambos sentidos del borde para encontrar el índice correcto.
                if (origin, destination) == (edge[0], edge[1]) or (origin, destination) == (edge[1], edge[0]):
                    # Actualiza el movimiento diario del borde sumando 'charge'.
                    edges_batch_full[i] = (origin, destination, {"Daily_movement": edges_batch_full[i][2]["Daily_movement"] + charge})
            continue
        else:
            # Si el borde no existe, agrega una nueva entrada en 'edges_batch_full'.
            edges_batch_full.append((edge[0], edge[1], {"Daily_movement": edge[2]["Daily_movement"]}))
    
    # Retorna 'edges_batch_full' después de actualizar o agregar los bordes.
    return edges_batch_full


def actualize_graph_carga(conn=conn):
    # Obtiene la lista de nodos y aristas desde la conexión proporcionada.
    nodes = conn.getVertices("Nodes")
    aristas = conn.getEdgesByType(edgeType="distribute_to")

    # Inicializa listas para almacenar nodos y aristas con datos escalados.
    nodes_batch_scale = []
    aristas_batch_scale = []
    
    # Inicializa una variable para registrar los estados de nodos y aristas.
    logs_state_nodes_and_arristas = ""

    # Contadores para los diferentes niveles de carga de nodos y aristas.
    node_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}
    arista_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}

    # Itera sobre cada nodo para determinar su nivel de carga.
    for node in nodes:
        node_capacidad = node["attributes"]["Capacity"]
        node_stock = node["attributes"]["Stock"]
        
        # Determina el nivel de carga del nodo basado en su capacidad y stock.
        if node_capacidad == 0:
            nivel_stock = "No definido"
        else:
            if (node_stock / node_capacidad) > 1:
                nivel_stock = "Sobrecargado"
                node_counts["Sobrecargado"] += 1
                logs_state_nodes_and_arristas += f"⚠️ {node['v_id']} es {nivel_stock} \n"
            elif 0.7 < (node_stock / node_capacidad) <= 1:
                nivel_stock = "Optimal"
                node_counts["Optimal"] += 1
            else:
                nivel_stock = "Subcargado"
                logs_state_nodes_and_arristas += f"⚠️ {node['v_id']} es {nivel_stock} \n"
                node_counts["Subcargado"] += 1

        # Añade el nodo con su nivel de carga a la lista de nodos a escalar.
        nodes_batch_scale.append((node["v_id"], {"Carga": nivel_stock}))

    # Itera sobre cada arista para determinar su nivel de carga.
    for arista in aristas:
        capacity_edge = arista["attributes"]["Capacity"]
        daily_movement = arista["attributes"]["Daily_movement"]
        
        # Determina el nivel de carga de la arista basado en su capacidad y movimiento diario.
        if capacity_edge == 0:
            nivel_arista = "No definido"
        else:
            if (daily_movement / capacity_edge) > 1:
                nivel_arista = "Sobrecargado"
                arista_counts["Sobrecargado"] += 1
                logs_state_nodes_and_arristas += f"⚠️ la arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} \n"
            elif 0.7 < (daily_movement / capacity_edge) <= 1:
                nivel_arista = "Optimal"
                arista_counts["Optimal"] += 1
            else:
                nivel_arista = "Subcargado"
                logs_state_nodes_and_arristas += f"⚠️ la arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} \n"
                arista_counts["Subcargado"] += 1

        # Añade la arista con su nivel de carga a la lista de aristas a escalar.
        aristas_batch_scale.append((arista["from_id"], arista["to_id"], {"Carga": nivel_arista}))

    # Actualiza los nodos en la base de datos con los datos escalados.
    conn.upsertVertices("Nodes", nodes_batch_scale)
    
    # Actualiza las aristas en la base de datos con los datos escalados.
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=aristas_batch_scale)

    # Si no se registraron cambios, indica que todos los nodos y aristas están en un estado óptimo.
    if logs_state_nodes_and_arristas == "":
        logs_state_nodes_and_arristas = "Todos los nodos y aristas están en un estado óptimo."

    # Crea un diccionario con los registros de estado y los conteos de nodos y aristas.
    results = {
        "logs": logs_state_nodes_and_arristas,
        "node_counts": node_counts,
        "arista_counts": arista_counts
    }

    # Retorna el diccionario con los resultados.
    return results


# Main transfer function

def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):

    # Lanza la función A* para encontrar el camino óptimo entre node_initial y node_final
    results_astar = conn.runInstalledQuery("tg_astar", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })
    
    # Obtiene el orden de los nodos en el camino encontrado
    order_taken = path_taken(results_astar)
    
    # Obtiene el conjunto de aristas involucradas en el camino
    edge_set = results_astar[2]['@@display_edge_set']
    
    # Reorganiza las aristas de acuerdo al orden de los nodos en el camino
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    # Obtiene el conjunto de nodos involucrados en el camino
    node_set = results_astar[2]['tmp']
    
    # Reorganiza los nodos de acuerdo al orden en el camino
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        # Recupera los atributos de los nodos inicial y final
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        # Verifica si hay suficiente inventario en el nodo inicial
        if charge > node_i_stock:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente inventario ({charge} > {node_i_stock})\n"
            log_list += error_message

        # Verifica si hay suficiente capacidad de carga en el nodo final
        if charge > node_f_load_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
            log_list += error_message

        # Verifica si hay suficiente capacidad de descarga en el nodo inicial
        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
            log_list += error_message

        # Recupera la capacidad de la arista y el movimiento diario actual
        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

        # Verifica si la arista puede soportar el movimiento adicional de carga
        if daily_movement + charge > capacity_edge:
            error_message = f"Enviar {charge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
            log_list += error_message

        # Actualiza los inventarios de los nodos inicial y final
        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        # Actualiza el movimiento diario en las aristas recorridas y acumula el costo
        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost += element["attributes"]["Price"]

        # Añade los nodos inicial y final a los conjuntos de nodos únicos
        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)
        
        # Incrementa el conteo de operaciones y el número total de palets
        operation_count += 1
        palets_number += charge

        return nodes_batch , edges_batch

    # Manejo de excepciones en caso de que ocurra algún error durante el proceso
    except Exception as e:
        log_list += f"Error durante la transferencia de {node_initial} a {node_final}: {str(e)}\n"
        return [], []

    except Exception as e:
        error_message = f"Error en la transferancia: {e}\n"
        log_list += error_message
        return error_message

import random 
import string
def generate_random_title(length=8):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

def generate_html():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="875" height="800" src="{tigergraph_insights_map}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html1():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="800" height="800" src="{tigergraph_insights_map1}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html800():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="700" height="800" src="{tigergraph_insights_map}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html1_800():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="700" height="800" src="{tigergraph_insights_map1}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """

def generate_html1000():
    random_title = generate_random_title()
    return f"""
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <iframe width="1000" height="800" src="{tigergraph_insights_map1}" title="{random_title}" frameborder="0"></iframe>
    </body>
    </html>
    """


def reset_html():
    new_html = generate_html()
    return gr.HTML(value=new_html, visible=True)

def reset_html1():
    new_html = generate_html1()
    return gr.HTML(value=new_html, visible=True)

def reset_html1_1000():
    new_html = generate_html1000()
    return gr.HTML(value=new_html, visible=True)


def reset_html_800():
    new_html = generate_html800()
    return gr.HTML(value=new_html, visible=True)

def reset_html1_800():
    new_html = generate_html1_800()
    return gr.HTML(value=new_html, visible=True)


# Thread safety
lock = threading.Lock()


def make_daily_movements(input_file, column_origin, column_destination, column_transfert):
    # Variables globales para llevar un conteo de operaciones y otras métricas.
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full, edges_batch_full, df
    # Inicializa las variables globales.
    operation_count = 0
    palets_number = 0
    palets_cost = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = " "
    nodes_batch_full = []
    edges_batch_full = []

    # Lee el archivo CSV en un DataFrame.
    df = pd.read_csv(input_file)

    # Función para realizar la transferencia de volumen entre nodos.
    def perform_transfer(row):
        global log_list
        try:
            # Obtiene el origen, destino y volumen de la fila del DataFrame.
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            
            # Llama a la función para obtener nodos y aristas actualizados.
            nodes_batch, edges_batch = transfert_nodes_and_edges(origin, destination, volume)
            
            # Usa un lock para asegurar que las actualizaciones a las listas sean seguras en un entorno multihilo.
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full, edges_batch, volume)
                
            # Incrementa el conteo de operaciones y otras métricas.
            global operation_count, palets_number, palets_cost
            operation_count += 1
            palets_number += volume
            palets_cost += volume * some_cost_per_palet  # Asumiendo que hay un costo por palet que debe definirse.
            unique_origin_nodes.add(origin)
            unique_destination_nodes.add(destination)
            
        except Exception as e:
            # Registra un error en caso de que la transferencia falle.
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    # Usa un ThreadPoolExecutor para procesar las transferencias en paralelo.
    with ThreadPoolExecutor(max_workers=1000) as executor:
        # Crea y envía tareas para cada fila del DataFrame.
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass  # Aquí se puede manejar el resultado de cada tarea si es necesario.

    # Actualiza los nodos y aristas en la base de datos con los datos acumulados.
    conn.upsertVertices("Nodes", nodes_batch_full)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full)
    
    # Actualiza el gráfico con el estado actual y obtiene el estado.
    state_log = actualize_graph_carga(conn)

    # Si no hubo errores, actualiza el mensaje de log.
    if log_list == " ":
        log_list = "All operations were successful."

    return (
        log_list,   #lista de los logs para el gradio
        reset_html(),  # esta función devuelve HTML para reiniciar el mapa tigergraph.
        gr.Button("Recargar mapa", elem_classes="otherbutton", visible=True),       #Elementos de Gradio
        gr.Number(label="KPI : Numero de operaciones", value=operation_count, visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Numero de palets", value=palets_number, visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Numero de origen", value=len(unique_origin_nodes), visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Destinaciones diferentes", value=len(unique_destination_nodes), visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Coste total", value=palets_cost, visible=True, elem_classes="dropdown"),
        gr.Number(label="KPI : Coste por palet", value=round(palets_cost / palets_number, 1), visible=True, elem_classes="dropdown")
    )

def make_daily_movements1(input_file, column_origin, column_destination, column_transfert):
    global operation_count1, palets_number1, palets_cost1, unique_origin_nodes1, unique_destination_nodes1, log_list1, nodes_batch_full1
    operation_count1 = 0
    palets_number1 = 0
    palets_cost1 = 0
    unique_origin_nodes1 = set()
    unique_destination_nodes1 = set()
    log_list1 = " "
    nodes_batch_full1 = []
    edges_batch_full1 = []

    df = pd.read_csv(input_file)

    def perform_transfer1(row):
        global log_list1
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            nodes_batch1, edges_batch1 = transfert_nodes_and_edges1(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full1, nodes_batch1, volume)
                add_or_update_final_edges_batch(edges_batch_full1, edges_batch1, volume)
        except Exception as e:
            error_message = f"{origin} to {destination}: No existe ninguna conexión. \n"
            log_list1 += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer1, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn1.upsertVertices("Nodes", nodes_batch_full1)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn1.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full1)
    results = actualize_graph_carga(conn)
    results1 = actualize_graph_carga(conn1)
    #Results looks like that : 
        # {"logs": logs_state_nodes_and_arristas,
        # "node_counts": node_counts,
        # "arista_counts": arista_counts}

    KPI_red_modificada = get_KPI(conn1, "distribute_to", "Nodes")

    return (log_list1,
        gr.Markdown("# Mapa red original",visible=True,elem_classes = "cabecero"),reset_html_800(),gr.Button("Recargar mapa",visible=True,elem_classes='otherbutton'),
        gr.Markdown("# Mapa red modificada",visible=True,elem_classes = "cabecero"),reset_html1_800(), gr.Button("Recargar mapa",visible=True,elem_classes='otherbutton'),
        # KPI RED ORIGINAL
        gr.Markdown("# Red original",visible=True,elem_classes='cabecero'),
        gr.Number(label="Cantidad de Aristas",value=numEdges,visible=True , elem_classes="dropdown"),                                 
        gr.Number(label="Cantidad de Nodos",value = numNodes,visible=True, elem_classes="dropdown"),
        gr.Number(label= "Capacidad media por arista",value = avg_capacity/2,visible=True, elem_classes="dropdown"),
        gr.Number(label="Capacidad total de las aristas ",value = total_capacity_edges/2,visible=True, elem_classes="dropdown"),
        gr.Number(label= "Capacidad total de los Almacenes",value = total_capacity_warehouse,visible=True, elem_classes="dropdown"),
        gr.Number(label="Stock total de los almacenes",value = total_stock,visible=True, elem_classes="dropdown"),
        # KPI ENTREGA ORIGINAL
        gr.Markdown("# Entrega original",visible=True,elem_classes='cabecero'),
        gr.Number(label="Numero de operaciones", value = operation_count,visible=True, elem_classes="dropdown"),                    
        gr.Number(label="Numero de palets",value = palets_number, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Numero de origen",value = len(unique_origin_nodes), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Destinaciones diferentes",value = len(unique_destination_nodes), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste total",value = palets_cost, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste por palet",value = round(palets_cost/palets_number,1), visible=True, elem_classes="dropdown"),
        # KPI RED SIMULADA
        gr.Markdown("# Red modificada",visible=True,elem_classes='cabecero'),
        gr.Number(label="Cantidad de Aristas", value=KPI_red_modificada["num_edges"], visible=True, elem_classes="dropdown"),
        gr.Number(label="Cantidad de Nodos", value=KPI_red_modificada["num_nodes"], visible=True, elem_classes="dropdown"),
        gr.Number(label="Capacidad media por arista", value=KPI_red_modificada["avg_capacity"], visible=True, elem_classes="dropdown"),
        gr.Number(label="Capacidad total de las aristas", value=KPI_red_modificada["total_capacity_edges"], visible=True, elem_classes="dropdown"),
        gr.Number(label="Capacidad total de los Almacenes", value=KPI_red_modificada["total_capacity_warehouses"], visible=True, elem_classes="dropdown"),
        gr.Number(label="Stock total de los almacenes", value=KPI_red_modificada["total_stock"], visible=True, elem_classes="dropdown"),
        # KPI ENTREGA SIMULADA
        gr.Markdown("# Entrega modificada",visible=True,elem_classes='cabecero'),
        gr.Number(label="Numero de operaciones", value = operation_count1,visible=True, elem_classes="dropdown"),                    
        gr.Number(label="Numero de palets",value = palets_number1, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Numero de origen",value = len(unique_origin_nodes1), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Destinaciones diferentes",value = len(unique_destination_nodes1), visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste total",value = palets_cost1, visible=True, elem_classes="dropdown"), 
        gr.Number(label="Coste por palet",value = round(palets_cost1/palets_number1,1), visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
        gr.Textbox(value=results["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=int(round(results["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
        gr.Textbox(value=int(round(results["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results1["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
        gr.Textbox(value=results1["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results1["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=int(round(results1["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
        gr.Textbox(value=int(round(results1["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown"),
        gr.Number(label = "Precio de la entrega mejorada de % ", value = int(((palets_cost-palets_cost1)/palets_cost)*100), visible=True, elem_classes="dropdown"),
        gr.Number(label="Porcentaje",elem_classes="dropdown",visible=True),
        gr.Button("Subir carga",elem_classes="otherbutton",visible=True),
        gr.Button("Bajar carga",elem_classes="otherbutton",visible=True)

        )




def transfert_nodes_and_edges1(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    global operation_count1, palets_number1, palets_cost1, unique_origin_nodes1, unique_destination_nodes1, log_list1, nodes_batch_full1
    results_astar = conn1.runInstalledQuery("tg_astar_test", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })
    
    order_taken = path_taken(results_astar)
    edge_set = results_astar[2]['@@display_edge_set']
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    node_set = results_astar[2]['tmp']
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        if charge > node_i_stock:
            error_message = f"{node_initial} hasta {node_final}: No hay suficientes inventario ({charge} > {node_i_stock})\n"
            log_list1 += error_message

        if charge > node_f_load_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de carga en el almacén de recepción (charge: {charge} > LoadCapacity: {node_f_load_capacity})\n"
            log_list1 += error_message

        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} hasta {node_final}: No hay suficiente capacidad de descarga desde el almacén de salida (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})\n"
            log_list1 += error_message

        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

        if daily_movement + charge > capacity_edge:
            error_message = f"Enviar {charge} palets desde {node_initial} hasta {node_final}: Capacidad de la arista insuficiente ({daily_movement + charge} > {capacity_edge})\n"
            log_list1 += error_message

        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost1 += element["attributes"]["Price"]

        unique_origin_nodes1.add(node_initial)
        unique_destination_nodes1.add(node_final)
        operation_count1 += 1
        palets_number1 += charge

        return nodes_batch , edges_batch

    except Exception as e:
        error_message = f"Error en la transferancia: {e} \n"
        log_list1 += error_message
        return error_message
# log_messages = make_daily_movements(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\entrega_demo.csv", "CODE_Origin", "CODE_Destination", "Palets")

# Funciones de modificacion de red

In [5]:
import json
global logs_modif_network
logs_modif_network = " "


# ADD NODES 

def add_node(id, attributes_dict=None, vertex_type="Nodes"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertVertex(vertex_type, id)
        message =  f"Nodo {id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertVertex(vertex_type, id, attributes=attributes)
        message =  f"Nodo {id} añadido with attributos : {attributes} \n"
        logs_modif_network +=message
    return logs_modif_network

def get_values_nodes(*args):
    values = {key: val for key, val in zip(atributes_nodes.keys(), args)}
    return values

def get_values_edges(*args):
    values = {key: val for key, val in zip(atributes_edges.keys(), args)}
    return values["Capacity"]


def add_node_with_values(id, *args):
    attributes = get_values_nodes(*args)
    attributes_json = json.dumps(attributes)
    actualize_graph_carga(conn)
    return add_node(id, attributes_json), reset_html1()



# ADD EDGES 
def add_new_edge(source_vertex_id, target_vertex_id, attributes_dict=None,source_vertex_type="Nodes",target_vertex_type = "Nodes", edge_type="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id)
        actualize_graph_carga(conn)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id}  añadida"
        logs_modif_network += message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        actualize_graph_carga(conn)
        conn1.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id, attributes=attributes)
        message =  f"Arista de {source_vertex_id} hasta {target_vertex_id} añadida con los atributos siguientes: {attributes} \n"
        logs_modif_network += message
    return logs_modif_network

def add_edge_with_values(source_vertex_id, target_vertex_id, *args):
    attributes = get_values_edges(*args)
    attributes_json = json.dumps(attributes)
    return add_new_edge(source_vertex_id,target_vertex_id, attributes_json), reset_html1()

# GET ATTRIBUTES
attributes_nodes_list = []
attributes_edges_list = []
conex_nodes_list = conn1.getVertexAttrs('Nodes')
conex_edges_list = conn1.getEdgeAttrs('distribute_to')

for node in conex_nodes_list:
    attributes_nodes_list.append(node[0])


for edge in conex_edges_list:
    attributes_edges_list.append(edge[0])


#Remove carga atributes as it is here only for a cheat with colors of the tigergraph map. no need to change that
if 'Carga' in attributes_nodes_list:
    attributes_nodes_list.remove('Carga')

if 'Carga' in attributes_edges_list:
    attributes_edges_list.remove('Carga')


# GET NODES NAMES
list_name_nodes = []
name_nodes_conn = conn1.getVertices("Nodes")
for i in range(0, len(name_nodes_conn)):
    list_name_nodes.append(name_nodes_conn[i]["v_id"])

def add_edge(source_vertex_id, target_vertex_id, attributes_dict=None, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id)
        message =  f"Arista desde {source_vertex_id} hasta {target_vertex_id} añadido\n"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn1.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id,
                        attributes=attributes)
        message =  f"Arista {source_vertex_id} hasta {target_vertex_id} añadido con los atributos siguientes : {attributes} \n"
        logs_modif_network +=message
        return logs_modif_network


#Erase Data

def erase_edge(source_vertex_id, target_vertex_id, vertex_type="Nodes", edgeType="distribute_to"):
    global logs_modif_network
    result = conn1.delEdges(sourceVertexType=vertex_type,
                           targetVertexType=vertex_type,
                           edgeType=edgeType,
                           sourceVertexId=source_vertex_id,
                           targetVertexId=target_vertex_id)
    
    if result == {edgeType: 0}:
        message = f"Arista de {source_vertex_id} hacia {target_vertex_id} no existe o no ha podido borrarse\n"
    else:
        message = f"Arista de  {source_vertex_id} hacia {target_vertex_id} borrada \n"
    
    logs_modif_network += message
    return logs_modif_network , reset_html1()

    


def erase_node(id, vertex_type="Nodes"):
    global logs_modif_network
    result = conn1.delVerticesById(vertexIds=id, vertexType=vertex_type)
    
    if result == 0:
        message = f"Nodo {id} no existe o no ha podido borrarse\n"
    else:
        message = f"Nodo {id} borrado\n"
    
    logs_modif_network += message
    return logs_modif_network , reset_html1()




# add_edge_with_values("Las Palmas de Gran Canaria","Seville",{"Capacity": 0, "Daily_movement": 0, "Price": 0})



# Tab modif cargas

In [6]:
def increase_charge(conn, percentage):
    aristas = conn.getEdgesByType(edgeType="distribute_to")
    new_charges = []
    for arista in aristas:
        current_charge = arista["attributes"]["Daily_movement"]
        new_charge = current_charge * (1 + percentage / 100)
        arista["attributes"]["Daily_movement"] = new_charge
        new_charges.append((arista["from_id"], arista["to_id"], {"Daily_movement": new_charge}))

    return new_charges


def decrease_charge(conn, percentage):
    aristas = conn.getEdgesByType(edgeType="distribute_to")
    new_charges = []
    logs_state_aristas = ""
    arista_counts = {"Sobrecargado": 0, "Subcargado": 0, "Optimal": 0}

    for arista in aristas:
        current_charge = arista["attributes"]["Daily_movement"]
        capacity_edge = arista["attributes"]["Capacity"]
        new_charge = current_charge * (1 - percentage / 100)
        arista["attributes"]["Daily_movement"] = new_charge
        
        if capacity_edge == 0:
            nivel_arista = "No definido"
        else:
            if (new_charge / capacity_edge) > 1:
                nivel_arista = "Sobrecargado"
                arista_counts["Sobrecargado"] += 1
                logs_state_aristas += f"⚠️ La arista {arista['from_id']} hasta {arista['to_id']} es {nivel_arista} con un movimiento diario de {new_charge}\n"
            elif 0.7 < (new_charge / capacity_edge) <= 1:
                nivel_arista = "Optimal"
                arista_counts["Optimal"] += 1
            else:
                nivel_arista = "Subcargado"
                arista_counts["Subcargado"] += 1

        new_charges.append((arista["from_id"], arista["to_id"], {"Daily_movement": new_charge, "Carga": nivel_arista}))

    if logs_state_aristas == "":
        logs_state_aristas = "Aucune arista n'est surchargée."

    results = {
        "logs": logs_state_aristas,
        "arista_counts": arista_counts,
    }

    return results , new_charges

def update_batch_increase(conn,percentage):
    batch = increase_charge(conn, percentage)
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=batch)

def update_batch_decrease(conn,percentage):
    logs , batch = decrease_charge(conn, percentage)
    conn.upsertEdges(sourceVertexType="Nodes", targetVertexType="Nodes", edgeType="distribute_to", edges=batch)

    # return(gr.Textbox(value=logs["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
    #     gr.Textbox(value=logs["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
    #     gr.Textbox(value=logs["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
    #     gr.Textbox(value=int(round(logs["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
    #     gr.Textbox(value=int(round(logs["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown"))



# a = update_batch_decrease(conn,10)

In [7]:
def make_daily_movements_with_percentages_positive(input_file, column_origin, column_destination, column_transfert,percentage):
    global operation_count3, palets_number3, palets_cost3, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full2,edges_batch_full2,df
    operation_count3 = 0
    palets_number3 = 0
    palets_cost3 = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = " "
    nodes_batch_full2 = []
    edges_batch_full2 = []

    df = pd.read_csv(input_file)

    def perform_transfer(row):
        global log_list
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            positive_or_negative = "positive"
            if positive_or_negative == "positive":
                volume = float(row[column_transfert]) * (percentage / 100)
            if positive_or_negative == "negative":
                volume = float(row[column_transfert]) * (1 - percentage / 100)
            nodes_batch, edges_batch = transfert_nodes_and_edges1(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full2, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full2, edges_batch, volume)
        except Exception as e:
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn1.upsertVertices("Nodes", nodes_batch_full2)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn1.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full2)
    results2 = actualize_graph_carga(conn1)

    if log_list == " ":
        log_list = "All operations were successful."

    return (reset_html1(),
        gr.Textbox(value=results2["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=int(round(results2["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
        gr.Textbox(value=int(round(results2["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown")
        )
    # return (log_list, reset_html(), gr.Button("Recargar mapa",elem_classes="otherbutton",visible=True),
    #     gr.Number(label="KPI : Numero de operaciones", value = operation_count,visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Numero de palets",value = palets_number, visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Numero de origen",value = len(unique_origin_nodes), visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Destinaciones diferentes",value = len(unique_destination_nodes), visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Coste total",value = palets_cost, visible=True,elem_classes="dropdown"), 
    #     gr.Number(label="KPI : Coste por palet",value = round(palets_cost/palets_number,1), visible=True,elem_classes="dropdown"))

def make_daily_movements_with_percentages_negative(input_file, column_origin, column_destination, column_transfert,percentage):
    global operation_count3, palets_number3, palets_cost3, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full2,edges_batch_full2,df
    operation_count3 = 0
    palets_number3 = 0
    palets_cost3 = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = " "
    nodes_batch_full2 = []
    edges_batch_full2 = []

    df = pd.read_csv(input_file)

    def perform_transfer(row):
        global log_list
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            positive_or_negative = "negative"
            if positive_or_negative == "positive":
                volume = float(row[column_transfert]) * (percentage / 100)
            if positive_or_negative == "negative":
                volume = float(row[column_transfert]) * (1 - percentage / 100)
            nodes_batch, edges_batch = transfert_nodes_and_edges1(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full2, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full2, edges_batch, volume)
        except Exception as e:
            error_message = f"{origin} hasta {destination}, no camino posible.\n"
            log_list += error_message

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn1.upsertVertices("Nodes", nodes_batch_full2)
    nodetype = "Nodes"
    edgetype = "distribute_to"
    conn1.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full2)
    results2 = actualize_graph_carga(conn1)

    if log_list == " ":
        log_list = "All operations were successful."

    return (reset_html1(),
        gr.Textbox(value=results2["logs"], label="Estado de la red", visible=True,lines=2,max_lines=10, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Sobrecargado"], label="Nodos sobrecargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=results2["node_counts"]["Subcargado"], label="Nodos subcargados", visible=True, elem_classes="dropdown"),
        gr.Textbox(value=int(round(results2["arista_counts"]["Sobrecargado"]/2,0)), label="Aristas sobrecargadas", visible=True, elem_classes="dropdown"),  #Divide by 2 to take into account the fact that the edges are bidirectional
        gr.Textbox(value=int(round(results2["arista_counts"]["Subcargado"]/2,0)), label="Aristas subcargadas", visible=True, elem_classes="dropdown")
        )

# a = make_daily_movements_with_percentages(r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\CSV_entrega\df_1_move.csv", "CODE_Origin", "CODE_Destination", "Palets",10,"positive")

In [8]:
def read_csv_path(input_file):
    df = pd.read_csv(input_file.name)
    target_column = sorted(list(df.columns))
    return  gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column)

Value first tab

In [9]:
name_edge = "distribute_to"
name_node = "Nodes"


#######RED ORIGINAL######

numEdges = conn.getEdgeCount(name_edge)
numNodes = conn.getVertexCount(name_node)
edge = conn.getEdgeStats(name_edge)
avg_capacity = round(edge[name_edge]['Capacity']['AVG'],1)

edges = conn.getEdgesByType(name_edge)
total_capacity_edges = 0
for warehouse in edges :
    capacity_edges = warehouse["attributes"]["Capacity"]     
    total_capacity_edges += round(capacity_edges,1)

almacenes = conn.getVertices(name_node)
total_capacity_warehouse = 0
for warehouse in almacenes :
    capacity_edges = warehouse["attributes"]["Capacity"]      
    total_capacity_warehouse += round(capacity_edges,1)
    total_capacity_warehouse = round(total_capacity_warehouse,1)

almacen = conn.getVertices(name_node)
total_stock = 0
for warehouse in almacen :
    stock = warehouse["attributes"]["Stock"]     
    total_stock += stock
    total_stock = round(total_stock,0)

capacity_per_edge = round(total_capacity_warehouse/numEdges,1)


#######RED SIMULADA######
import concurrent.futures

def get_num_edges(conn, edge_name):
    return conn.getEdgeCount(edge_name)

def get_num_nodes(conn, node_name):
    return conn.getVertexCount(node_name)

def get_avg_capacity(conn, edge_name):
    edges = conn.getEdgeStats(edge_name)
    avg_capacity = edges[edge_name]['Capacity']['AVG']
    return round(avg_capacity/2, 1)

def get_total_capacity_edges(conn, edge_name):
    edges = conn.getEdgesByType(edge_name)
    total_capacity = 0
    for edge in edges:
        total_capacity += edge["attributes"]["Capacity"]
    return total_capacity /2

def get_total_capacity_nodes(conn, node_name):
    warehouses = conn.getVertices(node_name)
    total_capacity = 0
    for warehouse in warehouses:
        total_capacity += warehouse["attributes"]["Capacity"]
    return round(total_capacity, 1)

def get_total_stock(conn, node_name):
    warehouses = conn.getVertices(node_name)
    total_stock = 0
    for warehouse in warehouses:
        total_stock += warehouse["attributes"]["Stock"]
    return round(total_stock, 0)

def get_capacity_per_edge(total_capacity_warehouses, num_edges):
    if num_edges == 0:
        return 0
    return round(total_capacity_warehouses / num_edges, 0)

def get_KPI(conn, edge_name, node_name):
     with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            "num_edges": executor.submit(get_num_edges, conn, edge_name),
            "num_nodes": executor.submit(get_num_nodes, conn, node_name),
            "avg_capacity": executor.submit(get_avg_capacity, conn, edge_name),
            "total_capacity_edges": executor.submit(get_total_capacity_edges, conn, edge_name),
            "total_capacity_warehouses": executor.submit(get_total_capacity_warehouses, conn, node_name),
            "total_stock": executor.submit(get_total_stock, conn, node_name)
        }

        results = {name: future.result() for name, future in futures.items()}
        results["capacity_per_edge"] = get_capacity_per_edge(results["total_capacity_warehouses"], results["num_edges"])
        
        return results

In [10]:
import gradio as gr

class GradioMetro:
    
    def front_func(self):
        with gr.Blocks(css="stylessupersimple.css", title ="Red de distribución",theme="Monochrome") as demo:
            gr.Markdown("## ""![](file/Gradio/logo_lis.svg) Red de distribución""", elem_classes="cabecero")
            with gr.Tabs() as tabs:

                with gr.TabItem("KPIs red",id=1):
                    with gr.Row():
                        with gr.Column(scale=5):
                            map_tiger = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="1000" height="800"
                                    src=""" + tigergraph_insights_map1 + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""")
                            btn_reload_map =  gr.Button("Recargar mapa",elem_classes="otherbutton")
                            btn_reload_map.click(fn=reset_html1_1000,outputs=[map_tiger])  
                        with gr.Column(scale=2):
                            gr.Number(label="KPI : Cantidad de Aristas",value=numEdges,elem_classes='textbox')
                            gr.Number(label="KPI : Cantidad de Nodos",value = numNodes,elem_classes='textbox')
                            gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity,elem_classes='textbox')
                            gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges,elem_classes='textbox')
                            gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse,elem_classes='textbox')
                            gr.Number(label="KPI : Stock total de los almacenes",value = total_stock,elem_classes='textbox')
                
                with gr.TabItem("Cargar movimientos",id=2):
                    with gr.Row():
                        with gr.Column(scale=5):
                            upload_file = gr.File(label="Cargar fichero", type="filepath",elem_classes='textbox')

                        with gr.Column(scale=4):
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True,elem_classes='textbox')
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True,elem_classes='textbox')
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True ,elem_classes='textbox')
                            load_data_send = gr.Button("Lanzar los envios",elem_classes="otherbutton")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])
                    with gr.Row():
                        logs = gr.Textbox(label="Logs",elem_classes="dropdown")

                    with gr.Row():
                        with gr.Column(scale=10):
                            map_entrega = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="800" height="800"
                                    src=""" + tigergraph_insights_map + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""", visible = False
                                    )
                            btn_reload_map2 = gr.Button("Recargar mapa",elem_classes="otherbutton",visible=False)
                            btn_reload_map2.click(fn=reset_html,outputs=[map_entrega])  
                        with gr.Column(scale=6):
                  
                            operation_number = gr.Number(label="KPI : Numero de operaciones", visible=False)
                            palets_number = gr.Number(label="KPI : Numero de palets", visible=False)
                            origin_number = gr.Number(label="KPI : Numero de origen", visible=False)
                            distinct_destinations = gr.Number(label="KPI : Destinaciones diferentes", visible=False)
                            total_cost = gr.Number(label="KPI : Coste total", visible=False)
                            total_cost_per_palet = gr.Number(label="KPI : Coste por palet", visible=False)
        
                        load_data_send.click(make_daily_movements,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity],
                                             outputs=[logs,map_entrega,btn_reload_map2,operation_number,palets_number,origin_number,distinct_destinations,total_cost,total_cost_per_palet])

                with gr.TabItem("Escenario What if", id=3):
                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("# Añadir/modificar nodo",elem_classes="cabecero")
                            new_node_id = gr.Textbox(label="Nombre nuevo nodo",elem_classes="dropdown")
                            for e in attributes_nodes_list:
                                with gr.Row():
                                    atributes_nodes[e] = gr.Number(label=e,elem_classes="dropdown")
                        
                            with gr.Row():
                                btn_modif_nodes = gr.Button("Añadir/modificar nodos",elem_classes="otherbutton")

                        with gr.Column():
                            gr.Markdown("# Añadir/modificar arista",elem_classes="cabecero")

                            source_vertex_id_edge = gr.Textbox(label="Nodo de origen",elem_classes="dropdown",elem_id = "source_vertex")
                            target_vertex_id_edge = gr.Textbox(label="Nodo de destino",elem_classes="dropdown")

                            for e in attributes_edges_list:
                                with gr.Row():
                                    atributes_edges[e] = gr.Number(label=e,elem_classes="dropdown")
                            btn_modif_edge = gr.Button("Añadir/modificar arista",elem_classes="otherbutton")

                        with gr.Column():
                            gr.Markdown("# Borrar nodo/arista",elem_classes="cabecero")
                            source_vertex_id = gr.Textbox(label="Nodo de origen",elem_classes="dropdown",container = True)
                            target_vertex_id = gr.Textbox(label="Nodo de destino (si es una arista)",elem_classes="dropdown",container = True)
                            btn_erase_node = gr.Button("Borrar nodo",elem_classes="otherbutton")
                            btn_erase_edge = gr.Button("Borrar arista",elem_classes="otherbutton")  

                    with gr.Row():
                        logs = gr.Textbox(label = "logs",elem_classes="dropdown")

                    with gr.Row():
                        with gr.Column(scale=5):
                            upload_file = gr.File(label="Cargar fichero", type="filepath",elem_classes='textbox')
                        with gr.Column(scale=4):
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            load_data_send1 = gr.Button("Lanzar los envios en la red modificada",elem_classes="otherbutton")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])

                    with gr.Row():
                        logs1 = gr.Textbox(label = "logs",elem_classes="dropdown")

                            
                    with gr.Row():
                        with gr.Column():
                            markdown1 = gr.Markdown("# KPIs red original",visible=False,elem_classes = "cabecero")
                            num_edges_number1 = gr.Number(label="KPI : Cantidad de Aristas",value=numEdges,visible=False)
                            num_nodes_number1 = gr.Number(label="KPI : Cantidad de Nodos",value = numNodes,visible=False)
                            capacity_average_number1 = gr.Number(label= "KPI : Capacidad media por arista",value = avg_capacity,visible=False)
                            total_capacity_edge_number1 = gr.Number(label="KPI : Capacidad total de las aristas ",value = total_capacity_edges,visible=False)
                            total_capacity_warehouse_number1 = gr.Number(label= "KPI : Capacidad total de los Almacenes",value = total_capacity_warehouse,visible=False)
                            total_stock_number1 = gr.Number(label="KPI : Stock total de los almacenes",value = total_stock,visible=False)
                            ####KPI ENTREGA RED ORGINAL####
                        with gr.Column():
                            markdown2 = gr.Markdown("# KPIs Entrega original",visible=False,elem_classes = "cabecero")
                            operation_number1 = gr.Number(label= "KPI : Numero de operaciones",visible=False)
                            palets_number1 = gr.Number(label= "KPI : Numero de palets",visible=False)
                            origin_number1 = gr.Number(label= "KPI : Numero de origen",visible=False)
                            distinct_destinations1 = gr.Number(label= "KPI : Destinaciones diferentes",visible=False)
                            total_cost1 = gr.Number(label= "KPI : Coste total",visible=False)
                            total_cost_per_palet1 = gr.Number(label= "KPI : Coste por palet",visible=False)
                        with gr.Column():
                            ###KPI RED MODIFICADA####
                            markdown1_simul =gr.Markdown("# KPIs red modificada",visible=False,elem_classes = "cabecero")
                            num_edges_number_simul = gr.Number(label="KPI : Cantidad de Aristas",value="numEdges1",visible=False)
                            num_nodes_number_simul = gr.Number(label="KPI : Cantidad de Nodos",value = "numNodes1",visible=False)
                            capacity_average_number_simul = gr.Number(label= "KPI : Capacidad media por arista",value = "avg_capacity1",visible=False)
                            total_capacity_edge_number_simul = gr.Number(label="KPI : Capacidad total de las aristas ",value = "total_capacity_edges1",visible=False)
                            total_capacity_warehouse_number_simul = gr.Number(label= "KPI : Capacidad total de los Almacenes",value = "total_capacity_warehouse1",visible=False)
                            total_stock_number_simul = gr.Number(label="KPI : Stock total de los almacenes",value = "total_stock1",visible=False)
                        with gr.Column():
                            markdown2_simul =gr.Markdown("# KPIs entrega red modificada",visible=False,elem_classes = "cabecero")
                            operation_number_simul = gr.Number(label= "KPI : Numero de operaciones",visible=False)
                            palets_number_simul = gr.Number(label= "KPI : Numero de palets",visible=False)
                            origin_number_simul = gr.Number(label= "KPI : Numero de origen",visible=False)
                            distinct_destinations_simul = gr.Number(label= "KPI : Destinaciones diferentes",visible=False)
                            total_cost_simul = gr.Number(label= "KPI : Coste total",visible=False)
                            total_cost_per_palet_simul = gr.Number(label= "KPI : Coste por palet",visible=False)
                    
                    with gr.Row():
                        diference_price = gr.Number(label=" porcentaje diferencia precio",value = avg_capacity,visible=False)



                    with gr.Row():
                        with gr.Column():
                            markdown_mapa_original =gr.Markdown("# Mapa red original",visible=False,elem_classes = "cabecero")
                            map_entrega_original = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                        initial-scale=1.0"></head><body><iframe width="800" height="800"
                                        src=""" + tigergraph_insights_map + """ 
                                        title="tigergarph insights" frameborder="0"></iframe></body></html>""",visible=False)
                            btn_reload_map_original = gr.Button("Recargar mapa",visible=False)

                        with gr.Column():
                            markdown_mapa_modificada =gr.Markdown("# Mapa modificada",visible=False,elem_classes = "cabecero")
                            map_entrega_simulacion = gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                        initial-scale=1.0"></head><body><iframe width="800" height="800"
                                        src=""" + tigergraph_insights_map1 + """ 
                                        title="tigergarph insights" frameborder="0"></iframe></body></html>""",visible=False)
                            btn_reload_map_simul = gr.Button("Recargar mapa",visible=False)
                    
                    with gr.Row():
                        valor_porcentaje = gr.Number(label="Porcentaje de subir/bajar la carga",visible=False)
                        
                    with gr.Row():
                        btn_subir_carga = gr.Button("Subir carga",elem_classes="otherbutton",visible=False)
                        btn_bajar_carga = gr.Button("Bajar carga",elem_classes="otherbutton",visible=False)

                    with gr.Row():
                        with gr.Column(scale=5):
                            logs_red_state_o = gr.Textbox(label="Estado de la red", visible=False,elem_classes="dropdown")

                            KPIo_node_sobrecargado = gr.Textbox(label="Nodos sobrecargados", visible=False,elem_classes="dropdown")
                            KPIo_node_subcargado = gr.Textbox(label="Nodos subcargados", visible=False,elem_classes="dropdown")
                            KPIo_arista_sobrecargada = gr.Textbox(label="Aristas sobrecargadas", visible=False,elem_classes="dropdown")
                            KPIo_arista_subcargada = gr.Textbox(label="Aristas subcargadas", visible=False,elem_classes="dropdown")
                        with gr.Column(scale=5):
                            logs_red_state_m = gr.Textbox(label="Estado de la red", visible=False,elem_classes="dropdown")
                            KPIm_node_sobrecargado = gr.Textbox(label="Nodos sobrecargados", visible=False,elem_classes="dropdown")
                            KPIm_node_subcargado = gr.Textbox(label="Nodos subcargados", visible=False,elem_classes="dropdown")
                            KPIm_arista_sobrecargada = gr.Textbox(label="Aristas sobrecargadas", visible=False,elem_classes="dropdown")
                            KPIm_arista_subcargada = gr.Textbox(label="Aristas subcargadas", visible=False,elem_classes="dropdown")


                        load_data_send1.click(make_daily_movements1,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity],
                                            outputs=[logs1,markdown_mapa_original,map_entrega_original,btn_reload_map_original,markdown_mapa_modificada,map_entrega_simulacion,btn_reload_map_simul,
                                                    markdown1,num_edges_number1,num_nodes_number1,capacity_average_number1,total_capacity_edge_number1,total_capacity_warehouse_number1,total_stock_number1,   #Red original
                                                    markdown2,operation_number1,palets_number1,origin_number1,distinct_destinations1,total_cost1,total_cost_per_palet1,   #Entrega original
                                                    markdown1_simul,num_edges_number_simul,num_nodes_number_simul,capacity_average_number_simul,total_capacity_edge_number_simul,total_capacity_warehouse_number_simul,total_stock_number_simul, #Red simulada
                                                    markdown2_simul,operation_number_simul,palets_number_simul,origin_number_simul,distinct_destinations_simul,total_cost_simul,total_cost_per_palet_simul #Entrega simulada 
                                                    ,logs_red_state_o,
                                                    KPIo_node_sobrecargado,KPIo_node_subcargado,KPIo_arista_sobrecargada,KPIo_arista_subcargada,
                                                    logs_red_state_m,
                                                    KPIm_node_sobrecargado,KPIm_node_subcargado,KPIm_arista_sobrecargada,KPIm_arista_subcargada,diference_price,
                                                    valor_porcentaje,btn_subir_carga,btn_bajar_carga],
                                                    scroll_to_output=True) 
                        
                        btn_subir_carga.click(make_daily_movements_with_percentages_positive,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity,valor_porcentaje],
                                              outputs=[map_entrega_simulacion,logs_red_state_m,KPIm_node_sobrecargado,KPIm_node_subcargado,KPIm_arista_sobrecargada,KPIm_arista_subcargada])
                        btn_bajar_carga.click(make_daily_movements_with_percentages_negative,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity,valor_porcentaje],
                                              outputs=[map_entrega_simulacion,logs_red_state_m,KPIm_node_sobrecargado,KPIm_node_subcargado,KPIm_arista_sobrecargada,KPIm_arista_subcargada])


                    inputs_edge = [source_vertex_id_edge,target_vertex_id_edge] + [atributes_edges[e] for e in attributes_edges_list]
                    
                    inputs_nodes = [new_node_id] + [atributes_nodes[e] for e in attributes_nodes_list]
                    
                    def add_edge_with_values_wrapper(source, target, *args):
                        attributes = {e: arg for e, arg in zip(attributes_edges_list, args)}
                        return add_edge_with_values(source, target, attributes)

                    btn_modif_nodes.click(fn=add_node_with_values, inputs=inputs_nodes, outputs=[logs,map_entrega_simulacion])
                    btn_modif_edge.click(fn=add_edge_with_values_wrapper, inputs=inputs_edge, outputs=[logs,map_entrega_simulacion])
                    btn_erase_node.click(fn=erase_node, inputs=[source_vertex_id], outputs=[logs,map_entrega_simulacion])
                    btn_erase_edge.click(fn=erase_edge, inputs=[source_vertex_id,target_vertex_id], outputs=[logs,map_entrega_simulacion])
                    btn_reload_map_original.click(fn=reset_html_800,outputs=[map_entrega_original])
                    btn_reload_map_simul.click(fn=reset_html1_800,outputs=[map_entrega_simulacion])

                demo.launch(max_threads=1000)
                
if __name__ == "__main__":

    lis = GradioMetro()
    lis.front_func()


    #Reactualizar los colores cuando se hace modif de arrista despues de la entrega

c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\gradio\routes.py:1150: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\fastapi\applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\gradio\routes.py:1150: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\fastapi\applications.py:4495: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  return self.router.on_event(event_type)


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\starlette\templating.py:178: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(
c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\starlette\templating.py:178: DeprecationWarning: The `name` is not the first parameter anymore. The first parameter should be the `Request` instance.
Replace `TemplateResponse(name, {"request": request})` by `TemplateResponse(request, name)`.
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\gradio\queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\gradio\route_utils.py"